# Simulation Demonstration

This notebook demonstrates the construction of a control system using the node based model, and the simulation of the system.

In [21]:
import sys
import torch
import torch.nn as nn
sys.path.append('../')

# Reference Signal Node

In [22]:
from humancompatible.interconnect.simulators.nodes.reference import ReferenceSignal

## Aggregator Node

Aggregator nodes are the only node that can accept a many to one connection. They will combine these signals into a single signal.

Logic is controlled by a logic class. For the aggregator node, this requires the following:
- `self.tensors`: a list of tensors
- `self.forward`: a function that takes in a list of signals (tensors) and returns a single signal


In [23]:
class AggregatorLogic1:
    def __init__(self):
        self.tensors = []
        
    def forward(self, tensors):
        if type(tensors) is not list:
            tensors = [tensors]
        self.tensors = tensors
        result = torch.sum(torch.stack([torch.sum(t.detach().clone().requires_grad_()) for t in self.tensors])).unsqueeze(dim=0)
        return result
        
class AggregatorLogic2:
    def __init__(self):
        self.tensors = []
        
    def forward(self, tensors):
        if type(tensors) is not list:
            tensors = [tensors]
        self.tensors = tensors
        return torch.sum(torch.stack([torch.sum(t) for t in self.tensors])).unsqueeze(dim=0)
        

In [24]:
from humancompatible.interconnect.simulators.nodes.aggregator import Aggregator

## Controller Node

#### Creating New Logic Classes for Controllers

##### Overview

To create a new logic class for use with the `Controller` class, follow these guidelines. Your logic class should define the behavior of a specific type of controller (e.g., PI, PID, MPC).

##### Requirements

Each logic class must have:

1. An `variables` attribute: A list of input variable names, provided by the input signal from the prior node.
2. An `self.tensors` attribute: a dictionary of tensors that are used in the logic, uses the `torch` library
3. An `forward(values)` method: Computes and returns the controller output.

##### Basic Structure

```python
class YourLogicClass:
    def __init__(self, param1=default1, param2=default2):
        self.variables = ["input1", "input2", ...]
        self.tensors = {"intpit1": torch.tensor([0.0], requires_grad=True), ...}
        # Initialize other necessary attributes

    def forward(self, values):
        # Compute the output based on input values
        # Update internal state if necessary
        return output
```
Any information needed to be stored between cycles can also be stored in the logic class. This can be done by adding attributes to the class and updating them in the `forward` method.

Prewritten logic classes can be found in the `interconnect/simulators/logic/controllers` directory. New logic classes should be added to this directory if they would be useful for other users.

In [25]:
from humancompatible.interconnect.simulators.nodes.controller import Controller
from humancompatible.interconnect.simulators.logic.controllers.ReLU_controller import ReLUControllerLogic

## Population Node

A node that represents a population of agents. Takes a logic object, along with a number of agents and the positive and negative feedback signals. This logic will be applied to each agent in the population. In order to use multiple types of agents, multiple population nodes can be used in parallel, and joined with an aggregator node.

In [26]:
from humancompatible.interconnect.simulators.nodes.population import Population

class AgentLogic:
    def __init__(self, offset=0.3):
        self.tensors = {"x": torch.tensor([0.0], requires_grad=True),
                        "pi": torch.tensor([0.0], requires_grad=True)}
        self.variables = ["pi"]
        self.probability = 0.5
        self.offset = offset

    def set_probability(self, probability):
        self.probability = probability

    def forward(self, values, number_of_agents):
        self.tensors["pi"] = values["pi"]
        f1_part = torch.sum(torch.bernoulli(torch.ones(number_of_agents) * self.probability))
        f2_part = number_of_agents - f1_part
        result = f1_part * self.tensors["pi"] + f2_part * (self.tensors["pi"]*5 - self.offset)
        return result


## Delay Node

In [27]:
from humancompatible.interconnect.simulators.nodes.delay import Delay

class DelayLogic:
    def __init__(self, alpha=0.3):
        self.tensors = {"x": torch.tensor([0.0], requires_grad=True)}
        self.tensors = {"s": torch.tensor([0.0], requires_grad=True)}
        self.variables = ["x"]
        self.alpha = alpha
        
    def forward(self, values):
        self.tensors["x"] = values["x"]
        self.tensors["s"] = self.tensors["x"] * self.alpha + self.tensors["s"].detach() * (1.0 - self.alpha)
        result = self.tensors["s"]
        return result

## Filter Node

Filter nodes take a logic  object. For the controller, this requires the following:

- `self.tensors`: a dictionary of tensors that are used in the logic, uses the `torch` library
- `self.variables`: a list of variables that are used in the logic, in the order that they are passed to the node
- `forward(values)` method: computes and returns filter's output

In [28]:
from humancompatible.interconnect.simulators.nodes.filter import Filter

class FilterLogic:
    def __init__(self):
        self.tensors = {"S": torch.tensor([0.0], requires_grad=True),
                        "K": torch.tensor([2.0], requires_grad=True)}
        self.variables = ["S"]
    
    def forward(self, values):
        self.tensors["S"] = values["S"]
        result = - (self.tensors["S"]) / self.tensors["K"]
        return result

### Creating the system

The system is composed of a number of nodes.

Single nodes can be added using the `system.add_node` method. Multiple nodes can be added using the `system.add_nodes` method, where a list of nodes is passed as an argument.

To connect nodes, the `system.connect_nodes` method is used. This method takes two arguments, the first being the source node and the second being the target node.

The start node, where the process begins, and the checkpoint node, where an iteration is considered complete, are added using the `system.add_start` and `system.add_checkpoint` methods, respectively.

In [29]:
from humancompatible.interconnect.simulators.simulation import Simulation
class Sim(Simulation):
    def __init__(self, reference_signal=8.0):
        super().__init__()
        
        refsig = ReferenceSignal(name="r")
        refsig.set_reference_signal(reference_signal)
        agg1 = Aggregator(name="A1", logic=AggregatorLogic1()) # Error detector
        agg2 = Aggregator(name="A2", logic=AggregatorLogic2()) # Sums population outputs
        cont = Controller(name="C", logic=ReLUControllerLogic())
        pop1 = Population(name="P1", logic=AgentLogic(), number_of_agents=20)
        pop2 = Population(name="P2", logic=AgentLogic(offset=0.4), number_of_agents=20)
        delay = Delay(name="Z", logic=DelayLogic())
        fil = Filter(name="F", logic=FilterLogic())
        self.system.add_nodes([refsig, agg1, agg2, cont, pop1, pop2, delay, fil])
        self.system.connect_nodes(refsig, agg1)
        self.system.connect_nodes(agg1, cont)
        self.system.connect_nodes(cont, pop1)
        self.system.connect_nodes(cont, pop2)
        self.system.connect_nodes(pop1, agg2)
        self.system.connect_nodes(pop2, agg2)
        self.system.connect_nodes(agg2, delay)
        self.system.connect_nodes(delay, fil)
        self.system.connect_nodes(fil, agg1)
        
        self.system.set_start_node(refsig)
        self.system.set_checkpoint_node(agg1)
        
        # Set learning node
        # cont.logic.load_state_dict(torch.load("../../interconnect/examples/weights/weights_basic_RELU.pth"))
        # model = cont.logic
        # self.system.set_learning_model(model)
        # self.system.set_loss_function(nn.MSELoss())
        # optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        # self.system.set_optimizer(optimizer)
        
        # Load controller logic
        cont.logic.load_state_dict(torch.load("../../interconnect/examples/weights/weights_basic_RELU.pth"))
        
sim = Sim(reference_signal=20.0)

### Rendering the system graph

Calling the `system.render_graph()` method will produce a visualisation of the control system. This is useful for ensuring that the system has been constructed according to the desired configuration.

The start node, shaded in green, indicates where the process will begin.

The checkpoint node, shaded in blue, indicates where the iteration count will be incremented. Typically, this will be the first node in the circular path of the system.

In [30]:
sim.plot.render_graph()

### Checking the validity of the system

This is automatically done prior to running the simulation, but it can be done manually by calling the `check_system` method.

In [31]:
validSystem = sim.system.check_system()
if validSystem:
    print("System is valid")
else:
    for e in validSystem:
        print(e)

### Running the simulation

In [32]:
sim.system.run(300, show_trace=False, show_loss=False)

### Plotting Node history

In [33]:
sim.plot.node_outputs(sim.system.get_node("A1"))

In [34]:
# sim.system.get_node("C").logic.state_dict()

### Tuned Populations Logic functions

In [35]:
# torch.save(sim.system.get_node("C").logic.state_dict(), "./weights_basic_RELU.pth") # SAVING MODEL TO FILE

In [36]:
# sim.plot.population_probabilities(xMin=-10, xMax=10)

In [37]:
sim.plot.runtimes()

### Contraction on average test

In [38]:
from tests.contractionTests.contraction_test import get_factor_from_list
import numpy as np
eps = 0.05
r = get_factor_from_list(reference_signals=np.array([-200, 200]), agent_probs=np.array([[eps, 1.0 - eps], [eps, 1.0 - eps]]), it=200, show_node_outputs_plot="A1", sim_class=Sim, inputs_node="A1", outputs_node="F")

### Get the contraction factor

In [39]:
print(f"Factor = {r}")

### Get the probability distribution estimations of the error after N iterations of the system

In [40]:
from humancompatible.interconnect.simulators.distribution import Distribution
dist = Distribution(sim_class=Sim, iterations=20, samples=100)
distributions = dist.get_distributions(h=1.9, reference_signals=np.array([4.0, 5.0, 20.0, 25.0]), x_min=-40, x_max=40, bins=100)